In [ ]:
pip install -U langgraph langchain-core langchain-openai langsmith


In [ ]:
pip install --upgrade langchain


In [ ]:
from typing import TypedDict, Dict, List

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain.schema import HumanMessage  




In [4]:
class AgentState(TypedDict):
    messages: List[BaseMessage]
    files: Dict[str, str]


In [5]:
@tool
def write_file(filename: str, content: str) -> str:
    """Write content to a virtual file"""
    return f"WRITE::{filename}::{content}"

@tool
def read_file(filename: str) -> str:
    """Read content from a virtual file"""
    return f"READ::{filename}"

@tool
def ls() -> str:
    """List all files"""
    return "LS"


In [6]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
You are a LangGraph agent.

Rules:
- Summarize each topic separately.
- Save each summary using write_file.
- Read saved files before producing the final answer.
- Use the virtual file system instead of memory.
"""
    ),
    MessagesPlaceholder(variable_name="messages"),
])


In [7]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_react_agent(
    model=llm,
    tools=[write_file, read_file, ls],
    prompt=prompt
)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17056\3635448530.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [28]:
def agent_node(state: AgentState):
    result = agent.invoke(state)

    new_files = dict(state["files"])
    new_messages = state["messages"] + result["messages"]

    for msg in result["messages"]:
        text = msg.content if hasattr(msg, "content") else ""

        if text.startswith("WRITE::"):
            _, name, content = text.split("::", 2)
            new_files[name] = content

        elif text.startswith("READ::"):
            _, name = text.split("::", 1)
            content = new_files.get(name, "File not found")
            new_messages.append(SystemMessage(content=content))

    return {
        "messages": new_messages,
        "files": new_files
    }


In [9]:
builder = StateGraph(AgentState)

builder.add_node("agent", agent_node)
builder.set_entry_point("agent")
builder.add_edge("agent", END)

graph = builder.compile()


In [33]:
input_prompt = """
Summarize these three topics separately and then give a combined summary:
1. Renewable Energy Sources
2. Climate Change Impact on Oceans
3. Innovations in Sustainable Agriculture
"""

initial_state: AgentState = {
    "messages": [
        HumanMessage(content=input_prompt)
    ],
    "files": {}
}

result = graph.invoke(initial_state)

print("FINAL ANSWER:\n")
print(result["messages"][-1].content)

print("\nFILES CREATED:\n")
for k, v in result["files"].items():
    print(f"--- {k} ---")
    print(v)


FINAL ANSWER:

Innovations in sustainable agriculture focus on practices that enhance productivity while minimizing environmental impact. Techniques include precision farming, organic farming, agroforestry, and the use of genetically modified organisms (GMOs) for resilience. These innovations aim to improve soil health, conserve water, and reduce chemical inputs. Sustainable agriculture also emphasizes local food systems and community involvement, contributing to food security and economic viability while addressing climate change challenges.

FILES CREATED:

--- renewable_energy_sources.txt ---
Renewable energy sources are energy types that are replenished naturally and can be harnessed without depleting resources. Key sources include solar, wind, hydroelectric, geothermal, and biomass. These energy forms are crucial in reducing greenhouse gas emissions, combating climate change, and promoting energy security. The transition to renewable energy is essential for sustainable development